In [1]:
import torch
import ssl
import matplotlib.pyplot as plt
from src.logger.logger_initializer import LoggerInitializer

ssl._create_default_https_context = ssl._create_unverified_context
device = 'cuda'
torch.__version__

LoggerInitializer().init()

[2023-04-17 01:15:19] 32616 root {logger_initializer-47} INFO - Current process ID: 32616


True

In [2]:
from src.data_loader.dataset_getter import DatasetGetter
dataset = DatasetGetter('../../bayesian_test/data', batch_size=12).get()

In [3]:
from src.models.bnn.pyro_resnet18_bnn_classifier import PyroResnet18BnnClassifier

classifier = PyroResnet18BnnClassifier(dataset=dataset, device=device).init()

In [4]:
result_df, result_dict = classifier.fit(num_epoch=2)
display(result_df)
print(result_dict)

[2023-04-17 01:15:46] 32616 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 0/2]: f1_score: 0.144 accuracy_score: 0.234 loss: 17506156140.900 
[2023-04-17 01:16:08] 32616 src.scoring.epoch_score_printer {epoch_score_printer-21} INFO - [Epoch 1/2]: f1_score: 0.197 accuracy_score: 0.281 loss: 16940517713.813 


,f1_score,accuracy_score,loss
0,0.143851,0.234375,1.750616e+10
1,0.197429,0.281250,1.694052e+10


{'best_loss': 16940517713.812565, 'best_f1_score': 0.19742856291528857, 'best_accuracy_score': 0.28125, 'best_loss_model': OrderedDict([('_resnet.bn1.weight', tensor([ 2.1427e-01,  2.4035e-01, -5.1096e-08,  5.0559e-01, -1.7851e-03,
         2.0052e-01,  4.3095e-01,  4.4679e-04,  2.4640e-01, -2.2905e-04,
         3.1078e-01,  2.0885e-01,  3.6444e-01,  4.6406e-03,  2.8628e-01,
         2.6181e-01,  2.2936e-01,  3.5884e-01,  4.5865e-01,  2.8549e-01,
         2.6544e-01,  2.8384e-01,  3.0434e-01,  2.0421e-01,  2.4889e-01,
         2.8515e-01,  2.9779e-01,  3.0763e-01,  3.8612e-01,  2.9650e-01,
         2.6910e-01,  2.0495e-01,  2.6836e-01,  3.6801e-01,  4.1347e-01,
         3.8224e-01, -1.2430e-02,  1.7826e-01, -1.2335e-02,  2.1907e-01,
         1.7979e-01,  2.3277e-01,  2.5052e-01,  2.6179e-01,  3.3014e-01,
         2.7624e-01,  2.1918e-01,  2.5795e-01, -4.6452e-04,  2.6989e-01,
         2.1503e-01,  3.0470e-01,  3.6825e-01,  2.4899e-01,  3.6286e-01,
         2.1080e-01,  2.3994e-01,  2.2

In [5]:
classifier.save_model(result_dict)

In [6]:
test_model_weights_path = '../../bayesian_refactoring/assets/model_weights/pyro_resnet18bnn_32616_f19b55c3-39b8-4872-982c-abc6f3770be4.pt'

In [7]:
test_results = torch.load(test_model_weights_path)

In [9]:
new_classifier = PyroResnet18BnnClassifier(
    dataset=dataset, device=device
).load_model(
    model_state_dict=test_results['best_f1_score_model'],
    pyro_state_dict=test_results['pyro_state_dict']
)

In [10]:
new_classifier.score(dataset.validate_dataloader)

{'f1_score': 0.1882016191554561, 'accuracy_score': 0.28063241106719367}